In [7]:
shaking <- function(solution, k) {
  n <- length(solution)
  for (i in 1:k) {
    idx <- sample(1:n, 2)
    solution[idx] <- solution[rev(idx)]
  }
  return(solution)
}


In [6]:
local_search <- function(bins, bin_capacity, binpacking_instance) {
  current_solution <- bins
  best_solution <- current_solution
  best_cost <- binpacking_instance$evaluate(current_solution)
  improved <- TRUE
  
  while (improved) {
    improved <- FALSE
    for (i in 1:(length(current_solution) - 1)) {
      for (j in (i + 1):length(current_solution)) {
        new_solution <- current_solution
        new_solution[c(i, j)] <- new_solution[c(j, i)]
        new_cost <- binpacking_instance$evaluate(new_solution)
        if (new_cost < best_cost) {
          best_solution <- new_solution
          best_cost <- new_cost
          improved <- TRUE
        }
      }
    }
    if (improved) {
      current_solution <- best_solution
    }
  }
  
  return(best_solution)
}

In [14]:
bfd <- function(item_sizes, bin_capacity) {
  sorted_items <- sort(item_sizes, decreasing = TRUE)  # Ordenar los ítems en orden decreciente
  bins <- list()
  
  for (item in sorted_items) {
    placed <- FALSE
    for (bin in bins) {
      if (sum(unlist(bin)) + item <= bin_capacity) {
        bin[[length(bin) + 1]] <- item
        placed <- TRUE
        break
      }
    }
    if (!placed) {
      bins[[length(bins) + 1]] <- list(item)
    }
  }
  
  return(bins)
}


In [15]:
BinPacking <- setRefClass(
  "BinPacking",
  fields = list(
    item_sizes = "numeric",
    bin_capacity = "numeric"
  ),
  methods = list(
    evaluate = function(permutation) {
      bin_capacities <- numeric() 
      current_bin_capacity <- 0  

      for (i in permutation) {
        item_size <- item_sizes[i]
        if (current_bin_capacity + item_size <= bin_capacity) {
          current_bin_capacity <- current_bin_capacity + item_size
        } else {
          bin_capacities <- c(bin_capacities, current_bin_capacity) 
          current_bin_capacity <- item_size  
        }
      }
      bin_capacities <- c(bin_capacities, current_bin_capacity)  
      return(length(bin_capacities)) 
    },
    bfd = function() {
      sorted_items <- sort(item_sizes, decreasing = TRUE)  # Ordenar los ítems en orden decreciente
      bins <- list()  # Lista para almacenar los contenedores
      
      for (item in sorted_items) {
        placed <- FALSE
        # Intentar colocar el ítem en uno de los contenedores existentes
        for (bin in bins) {
          if (sum(unlist(bin)) + item <= bin_capacity) {
            bin[[length(bin) + 1]] <- item
            placed <- TRUE
            break
          }
        }
        # Si no se puede colocar el ítem en ningún contenedor existente, abrir uno nuevo
        if (!placed) {
          bins[[length(bins) + 1]] <- list(item)
        }
      }
      
      return(bins)
    }
  )
)


In [2]:
read_bin_packing_instance <- function(filename) {
  lines <- readLines(filename)
  P <- as.integer(lines[1])  # Número de instancias de problemas
  problems <- list()
  index <- 2
  for (i in 1:P) {
    problem_id <- trimws(lines[index])
    index <- index + 1
    bin_info <- strsplit(trimws(lines[index]), "\\s+")[[1]]
    bin_capacity <- as.numeric(bin_info[1])
    num_items <- as.integer(bin_info[2])
    best_known_solution <- as.integer(bin_info[3])
    index <- index + 1
    items <- numeric(num_items)
    for (j in 1:num_items) {
      items[j] <- as.numeric(trimws(lines[index]))
      index <- index + 1
    }
    problems[[i]] <- list(
      problem_id = problem_id,
      bin_capacity = bin_capacity,
      num_items = num_items,
      best_known_solution = best_known_solution,
      items = items
    )
  }
  return(problems)
}

In [9]:
vns <- function(binpacking_instance, max_iter = 100, max_no_improve = 10) {
  initial_solution <- sample(1:length(binpacking_instance$item_sizes))
  best_solution <- initial_solution
  best_cost <- binpacking_instance$evaluate(best_solution)
  no_improve <- 0
  
  for (iter in 1:max_iter) {
    k <- 1
    while (k <= max_no_improve) {
      new_solution <- shaking(best_solution, k)
      new_solution <- local_search(new_solution, binpacking_instance$bin_capacity, binpacking_instance)
      new_cost <- binpacking_instance$evaluate(new_solution)
      
      if (new_cost < best_cost) {
        best_solution <- new_solution
        best_cost <- new_cost
        no_improve <- 0
        k <- 1
      } else {
        k <- k + 1
        no_improve <- no_improve + 1
      }
      
      if (no_improve >= max_no_improve) break
    }
  }
  
  return(best_solution)
}

In [16]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)

Warning message in readLines(filename):
"incomplete final line found on './instances/binpack1.txt'"


In [20]:
instance <- instances[[2]]  # Seleccionar la primera instancia para este ejemplo
binpacking_instance <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)
best_solution <- vns(binpacking_instance, 100, 20)
print(best_solution)  
print(paste("Number of bins:", binpacking_instance$evaluate(best_solution)))
beepr::beep()

  [1] 103 118  39  18  99 105  33  12  16  10 106  29  51  30 101  40  46  15
 [19]  43  79  25 102  57  81  91  22  89  49  72  74  90  48  69 113 116   1
 [37]  21   9  84  97  27  23   7  87   2   4  11  93 108  41  75 100  62  24
 [55]  14  95  73  88  92  47  34  36  31 110  96  19  50  26   8  80  32  13
 [73]  38 112  76 115  61  77  56   5 104 109 107  63  66  52  94  53  85  54
 [91]  64  65  20  70  55   3 119 120  78  42  83  45  17   6  71  28  37  35
[109]  60  67  86  44 114 117  98  68  59  58  82 111
[1] "Number of bins: 51"
